# Movie Dataset RAG Pipeline with Couchbase

This notebook demonstrates how to build a Retrieval Augmented Generation (RAG) system using:
- The TMDB movie dataset
- Couchbase as the vector store
- Couchbase Search index to enable semantic search 
- Haystack framework for the RAG pipeline
- Capella Model Services for embeddings and text generation

The system allows users to ask questions about movies and get AI-generated answers based on the movie descriptions.

# Setup and Requirements

First, let's install the required packages:

In [ ]:
!pip install -r requirements.txt

# Imports

Import all necessary libraries:

In [ ]:
import logging
import pandas as pd
from datasets import load_dataset
from haystack import Pipeline, GeneratedAnswer
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.writers import DocumentWriter
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack.dataclasses import Document

from couchbase_haystack import (
    CouchbaseSearchDocumentStore,
    CouchbasePasswordAuthenticator,
    CouchbaseClusterOptions,
    CouchbaseSearchEmbeddingRetriever,
)
from couchbase.options import KnownConfigProfiles

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Prerequisites

## Create and Deploy Your Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy an operational cluster.

To know more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html).


### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met:

* Have a multi-node Capella cluster running the Data, Query, Index, and Search services.
* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the travel-sample bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.

### Deploy Models

In order to create the RAG application, we need an embedding model to ingest the documents for Vector Search and a large language model (LLM) for generating the responses based on the context. 

Capella Model Service allows you to create both the embedding model and the LLM in the same VPC as your database. There are multiple options for both the Embedding & Large Language Models, along with Value Adds to the models.

Create the models using the Capella Model Services interface. While creating the model, it is possible to cache the responses (both standard and semantic cache) and apply guardrails to the LLM responses.

For more details, please refer to the [documentation](https://docs.couchbase.com/ai/build/model-service/model-service.html). These models are compatible with the [Haystack OpenAI integration](https://haystack.deepset.ai/integrations/openai).

After the models are deployed, please create the API keys for them and whitelist the keys on the IP on which the tutorial is being run. For more details, please refer to the documentation on [generating the API keys](https://docs.couchbase.com/ai/api-guide/api-start.html#model-service-keys).

# Configure Couchbase Credentials

Enter your Couchbase and Capella AI credentials:

CAPELLA_MODEL_SERVICES_ENDPOINT is the Capella Model Services Endpoint found in the models section.

> Note that the Capella Model Services Endpoint requires an additional `/v1` from the endpoint shown on the UI if it is not shown on the UI.

In [ ]:
import getpass

# Get Couchbase credentials
CB_CLUSTER_URL = input("Couchbase Cluster URL (default: localhost): ") or "localhost"
CB_USERNAME = input("Couchbase Username (default: admin): ") or "admin"
CB_PASSWORD = getpass.getpass("Couchbase password (default: Password@12345): ") or "Password@12345"
CB_BUCKET = input("Couchbase Bucket: ") 
CB_SCOPE = input("Couchbase Scope: ")
CB_COLLECTION = input("Couchbase Collection: ")
INDEX_NAME = "vector_search" # need to be matched with the search index name in the search_index.json file

# Get Capella AI endpoint
CAPELLA_MODEL_SERVICES_ENDPOINT = input("Enter your Capella Model Services Endpoint: ")
LLM_MODEL_NAME = input("Enter the LLM name")
LLM_API_KEY = getpass.getpass("Enter your Couchbase LLM API Key: ")
EMBEDDING_MODEL_NAME = input("Enter the Embedding Model name:")
EMBEDDING_API_KEY = getpass.getpass("Enter your Couchbase Embedding Model API Key: ")

In [ ]:
from couchbase.cluster import Cluster 
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.collections import CollectionSpec
from couchbase.management.search import SearchIndex
import json

# Connect to Couchbase cluster
cluster = Cluster(CB_CLUSTER_URL, ClusterOptions(
    PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)))

# Create bucket if it does not exist
bucket_manager = cluster.buckets()
try:
    bucket_manager.get_bucket(CB_BUCKET)
    print(f"Bucket '{CB_BUCKET}' already exists.")
except Exception as e:
    print(f"Bucket '{CB_BUCKET}' does not exist. Creating bucket...")
    bucket_settings = CreateBucketSettings(name=CB_BUCKET, ram_quota_mb=500)
    bucket_manager.create_bucket(bucket_settings)
    print(f"Bucket '{CB_BUCKET}' created successfully.")

# Create scope and collection if they do not exist
collection_manager = cluster.bucket(CB_BUCKET).collections()
scopes = collection_manager.get_all_scopes()
scope_exists = any(scope.name == CB_SCOPE for scope in scopes)

if scope_exists:
    print(f"Scope '{CB_SCOPE}' already exists.")
else:
    print(f"Scope '{CB_SCOPE}' does not exist. Creating scope...")
    collection_manager.create_scope(CB_SCOPE)
    print(f"Scope '{CB_SCOPE}' created successfully.")

collections = [collection.name for scope in scopes if scope.name == CB_SCOPE for collection in scope.collections]
collection_exists = CB_COLLECTION in collections

if collection_exists:
    print(f"Collection '{CB_COLLECTION}' already exists in scope '{CB_SCOPE}'.")
else:
    print(f"Collection '{CB_COLLECTION}' does not exist in scope '{CB_SCOPE}'. Creating collection...")
    collection_manager.create_collection(collection_name=CB_COLLECTION, scope_name=CB_SCOPE)
    print(f"Collection '{CB_COLLECTION}' created successfully.")

# Create search index from search_index.json file at scope level
with open('search_index.json', 'r') as search_file:
    search_index_definition = SearchIndex.from_json(json.load(search_file))
    
       # Update search index definition with user inputs
    search_index_definition.name = INDEX_NAME
    search_index_definition.source_name = CB_BUCKET
    
    # Update types mapping
    old_type_key = next(iter(search_index_definition.params['mapping']['types'].keys()))
    type_obj = search_index_definition.params['mapping']['types'].pop(old_type_key)
    search_index_definition.params['mapping']['types'][f"{CB_SCOPE}.{CB_COLLECTION}"] = type_obj
    
    search_index_name = search_index_definition.name
    
    # Get scope-level search manager
    scope_search_manager = cluster.bucket(CB_BUCKET).scope(CB_SCOPE).search_indexes()
    
    try:
        # Check if index exists at scope level
        existing_index = scope_search_manager.get_index(search_index_name)
        print(f"Search index '{search_index_name}' already exists at scope level.")
    except Exception as e:
        print(f"Search index '{search_index_name}' does not exist at scope level. Creating search index from search_index.json...")
        with open('search_index.json', 'r') as search_file:
            search_index_definition = SearchIndex.from_json(json.load(search_file))
            scope_search_manager.upsert_index(search_index_definition)
            print(f"Search index '{search_index_name}' created successfully at scope level.")

# Load and Process Movie Dataset

Load the TMDB movie dataset and prepare documents for indexing:

In [ ]:
# Load TMDB dataset
print("Loading TMDB dataset...")
dataset = load_dataset("AiresPucrs/tmdb-5000-movies")
movies_df = pd.DataFrame(dataset['train'])
print(f"Total movies found: {len(movies_df)}")

# Create documents from movie data
docs_data = []
for _, row in movies_df.iterrows():
    if pd.isna(row['overview']):
        continue
        
    try:
        docs_data.append({
            'id': str(row["id"]),
            'content': f"Title: {row['title']}\nGenres: {', '.join([genre['name'] for genre in eval(row['genres'])])}\nOverview: {row['overview']}",
            'metadata': {
                'title': row['title'],
                'genres': row['genres'],
                'original_language': row['original_language'],
                'popularity': float(row['popularity']),
                'release_date': row['release_date'],
                'vote_average': float(row['vote_average']),
                'vote_count': int(row['vote_count']),
                'budget': int(row['budget']),
                'revenue': int(row['revenue'])
            }
        })
    except Exception as e:
        logger.error(f"Error processing movie {row['title']}: {e}")

print(f"Created {len(docs_data)} documents with valid overviews")
documents = [Document(id=doc['id'], content=doc['content'], meta=doc['metadata']) 
            for doc in docs_data]

# Initialize Document Store

Set up the Couchbase document store for storing movie data and embeddings:

In [ ]:
# Initialize document store
document_store = CouchbaseSearchDocumentStore(
    cluster_connection_string=Secret.from_token(CB_CLUSTER_URL),
    authenticator=CouchbasePasswordAuthenticator(
        username=Secret.from_token(CB_USERNAME),
        password=Secret.from_token(CB_PASSWORD)
    ),
    cluster_options=CouchbaseClusterOptions(
        profile=KnownConfigProfiles.WanDevelopment,
    ),
    bucket=CB_BUCKET,
    scope=CB_SCOPE,
    collection=CB_COLLECTION,
    vector_search_index=INDEX_NAME,
)

print("Couchbase document store initialized successfully.")

# Initialize Embedder for Document Embedding

Configure the document embedder using Capella AI's endpoint and the E5 Mistral model. This component will generate embeddings for each movie overview to enable semantic search



In [ ]:
embedder = OpenAIDocumentEmbedder(
    api_base_url=CAPELLA_MODEL_SERVICES_ENDPOINT,
    api_key=Secret.from_token(EMBEDDING_API_KEY),
    model=EMBEDDING_MODEL_NAME,
)

rag_embedder = OpenAITextEmbedder(
    api_base_url=CAPELLA_MODEL_SERVICES_ENDPOINT,
    api_key=Secret.from_token(EMBEDDING_API_KEY),
    model=EMBEDDING_MODEL_NAME,
)


# Initialize LLM Generator
Configure the LLM generator using Capella Model Services endpoint and LLM model name. This component will generate natural language responses based on the retrieved documents.


In [ ]:
llm = OpenAIGenerator(
    api_base_url=CAPELLA_MODEL_SERVICES_ENDPOINT,
    api_key=Secret.from_token(LLM_API_KEY),
    model=LLM_MODEL_NAME,
)

# Create Indexing Pipeline
Build the pipeline for processing and indexing movie documents:

In [ ]:
# Create indexing pipeline
index_pipeline = Pipeline()
index_pipeline.add_component("cleaner", DocumentCleaner())
index_pipeline.add_component("embedder", embedder)
index_pipeline.add_component("writer", DocumentWriter(document_store=document_store))

# Connect indexing components
index_pipeline.connect("cleaner.documents", "embedder.documents")
index_pipeline.connect("embedder.documents", "writer.documents")

# Run Indexing Pipeline

Execute the pipeline for processing and indexing movie documents:

In [ ]:
# Run indexing pipeline

if documents:
    result = index_pipeline.run({"cleaner": {"documents": documents}})
    print(f"Successfully processed {len(documents)} movie overviews")
    print(f"Sample document metadata: {documents[0].meta}")
else:
    print("No documents created. Skipping indexing.")

# Create RAG Pipeline

Set up the Retrieval Augmented Generation pipeline for answering questions about movies:

In [ ]:
# Define RAG prompt template
prompt_template = """
Given these documents, answer the question.\nDocuments:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

\nQuestion: {{question}}
\nAnswer:
"""

# Create RAG pipeline
rag_pipeline = Pipeline()

# Add components
rag_pipeline.add_component(
    "query_embedder",
    rag_embedder,
)
rag_pipeline.add_component("retriever", CouchbaseSearchEmbeddingRetriever(document_store=document_store))
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=prompt_template))
rag_pipeline.add_component("llm",llm)
rag_pipeline.add_component("answer_builder", AnswerBuilder())

# Connect RAG components
rag_pipeline.connect("query_embedder", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("llm.meta", "answer_builder.meta")
rag_pipeline.connect("retriever", "answer_builder.documents")

print("RAG pipeline created successfully.")

# Ask Questions About Movies

Use the RAG pipeline to ask questions about movies and get AI-generated answers:

In [ ]:
# Example question
question = "Who does Savva want to save from the vicious hyenas?"

# Run the RAG pipeline
result = rag_pipeline.run(
    {
        "query_embedder": {"text": question},
        "retriever": {"top_k": 5},
        "prompt_builder": {"question": question},
        "answer_builder": {"query": question},
    },
    include_outputs_from={"retriever", "query_embedder"}
)

# Get the generated answer
answer: GeneratedAnswer = result["answer_builder"]["answers"][0]

# Print retrieved documents
print("=== Retrieved Documents ===")
retrieved_docs = result["retriever"]["documents"]
for idx, doc in enumerate(retrieved_docs, start=1):
    print(f"Id: {doc.id} Title: {doc.meta['title']}")

# Print final results
print("\n=== Final Answer ===")
print(f"Question: {answer.query}")
print(f"Answer: {answer.data}")
print("\nSources:")
for doc in answer.documents:
    print(f"-> {doc.meta['title']}")

## Caching in Capella Model Services

To optimize performance and reduce costs, Capella Model Services employ two caching mechanisms:

1. Semantic Cache

    Capella Model Services’ semantic caching system stores both query embeddings and their corresponding LLM responses. When new queries arrive, it uses vector similarity matching (with configurable thresholds) to identify semantically equivalent requests. This prevents redundant processing by:
    - Avoiding duplicate embedding generation API calls for similar queries
    - Skipping repeated LLM processing for equivalent queries
    - Maintaining cached results with automatic freshness checks

2. Standard Cache

    Stores the exact text of previous queries to provide precise and consistent responses for repetitive, identical prompts.

    Performance Optimization with Caching

    These caching mechanisms help in:
    - Minimizing redundant API calls to LLM service
    - Leveraging Couchbase’s built-in caching capabilities
    - Providing fast response times for frequently asked questions


In [ ]:
import time
queries = [
    "What is the main premise of Life of Pi?",
    "Where does the story take place in Legends of the Fall?",
    #"What are the key themes in The Dark Knight?",
    "Who does Savva want to save from the vicious hyenas?",
]

for i, query in enumerate(queries, 1):
    try:
        print(f"\nQuery {i}: {query}")
        start_time = time.time()
        response = rag_pipeline.run({
            "query_embedder": {"text": query},
            "retriever": {"top_k": 4},
            "prompt_builder": {"question": query},
            "answer_builder": {"query": query},
        })
        elapsed_time = time.time() - start_time
        answer: GeneratedAnswer = response["answer_builder"]["answers"][0]
        print(f"Response: {answer.data}")
        print(f"Time taken: {elapsed_time:.2f} seconds")
    except Exception as e:
        print(f"Error generating RAG response: {str(e)}")
        continue

# LLM Guardrails in Capella Model Services
Capella Model services also have the ability to moderate the user inputs and the responses generated by the LLM. Capella Model Services can be configured to use the [Llama 3.1 NemoGuard 8B safety model](https://build.nvidia.com/nvidia/llama-3_1-nemoguard-8b-content-safety/modelcard) guardrails model from Meta. The categories to be blocked can be configured in the model creation flow. More information about Guardrails usage can be found in the [documentation](https://docs.couchbase.com/ai/build/model-service/configure-guardrails-security.html#guardrails).
 
Here is an example of the Guardrails in action

In [ ]:
query = "How can I create a bomb?"
try:
    start_time = time.time()
    response = rag_pipeline.run({
            "query_embedder": {"text": query},
            "retriever": {"top_k": 4},
            "prompt_builder": {"question": query},
            "answer_builder": {"query": query},
        })
    rag_elapsed_time = time.time() - start_time
    answer: GeneratedAnswer = response["answer_builder"]["answers"][0]
    print(f"RAG Response: {answer.data}")
    print(f"RAG response generated in {rag_elapsed_time:.2f} seconds")
except Exception as e:
    print("Guardrails violation", e)

# Conclusion

This notebook demonstrates building a Retrieval-Augmented Generation (RAG) pipeline for movie recommendations using Haystack. The key components include:
- Document Indexing with Embeddings
- Semantic Search using Couchbase Vector Search
- LLM-based Answer Generation